# 撈取資料庫特定條件資料

用途：統計每個頻道標籤（包含屬性標籤、內容標籤）的聲量數

## 操作流程

1. 安裝套件

    `pip install pyodbc`：讓 Python 可以和資料庫做連線

    `pip install python-dateutil`：比內建好用的時間運算套件
    
    `pip install pandas`：從輸入 sql 檔案、資料操作、到最後輸出 excel 檔案都需要用到的資料處理套件
2. 連線爬文資料庫並進行登入
3. 給定資料庫名稱（和網站 ID，非必要）與查詢起迄時間
4. 回傳所有標籤在每個月份的資料筆數 (dataframe)

In [ ]:
import pyodbc
import pandas as pd
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
from tqdm.notebook import trange, tqdm #可以出現進度條的模組

# 連線資料庫

In [ ]:
# 登入資訊
user = 'mingyanxxx' #機敏記得更改
password = 'giraffexxxxxxxxxxxxxx' #機敏記得更改

# 以 MariaDB 的驅動程式串 DB 資料庫
conn = pyodbc.connect(
    f'DRIVER={{MariaDB ODBC 3.1 Driver}};User={user};Password={password};Server=dc-data11.eland.com.tw;Port=3306;'
)

# 統計標籤聲量

### 函數一：統計整個資料庫當中的所有網站、頻道

In [ ]:
def get_tag_data_by_db(db_name, time_1, time_2):
    df = pd.DataFrame()
    start_time = time_1
    while True:
        if start_time < time_2:
            year_month = dt.strftime(dt.strptime(start_time, '%Y-%m-%d'), '%Y/%m')  # 為了輸出方便
            end_time = dt.strftime(dt.strptime(start_time, '%Y-%m-%d') + relativedelta(months=1), '%Y-%m-%d')  # 為了逐月計算

            sql_script = """
                            SELECT tagname_ch.name, SUM(a.count_v) FROM wh_query.channel_tag_mapping AS tagname
                            LEFT JOIN (
                                SELECT volume.s_area_id AS s_area_id, COUNT(volume.id) AS count_v FROM %s.ts_page_content AS volume
                                WHERE volume.post_time >= '%s'
                                AND volume.post_time < '%s'
                                GROUP BY s_area_id
                                ) AS a
                            ON tagname.s_area_id COLLATE UTF8MB4_GENERAL_CI = a.s_area_id
                            LEFT JOIN wh_query.channel_tag AS tagname_ch ON tagname.tag_id = tagname_ch.id
                            GROUP BY tagname_ch.name;
                        """ % (db_name, start_time, end_time)
            temp_query = pd.read_sql(sql_script, conn)  # 執行上方 SQL 指令並存起來
            temp_query['月份'] = dt.strftime(dt.strptime(start_time, '%Y-%m-%d'), '%Y/%m')  # 加入月份欄位
            df = pd.concat([df, temp_query], ignore_index=True)  # 將這次的結果與先前的結果合併起來
            print(f'資料庫 {db_name} 在 {year_month} 的資料已運算完成。')  # 顯示目前執行進度
            start_time = end_time  # 將開始時間取代為下一個月
        else:
            break
    df.columns = ['標籤名稱', '聲量數', '月份']
    return df

### 函數二：僅統計某特定網站

In [ ]:
def get_tag_data_by_site(db_name, s_id, time_1, time_2):
    df = pd.DataFrame()
    start_time = time_1
    while True:
        if start_time < time_2:
            year_month = dt.strftime(dt.strptime(start_time, '%Y-%m-%d'), '%Y/%m')  # 為了輸出方便
            end_time = dt.strftime(dt.strptime(start_time, '%Y-%m-%d') + relativedelta(months=1), '%Y-%m-%d')  # 為了逐月計算

            sql_script = """
                            SELECT tagname_ch.name, SUM(a.count_v) FROM wh_query.channel_tag_mapping AS tagname
                            LEFT JOIN (
                                SELECT volume.s_area_id AS s_area_id, COUNT(volume.id) AS count_v FROM %s.ts_page_content AS volume
                                WHERE volume.post_time >= '%s'
                                AND volume.post_time < '%s'
                                AND s_id = '%s'
                                GROUP BY s_area_id
                                ) AS a
                            ON tagname.s_area_id COLLATE UTF8MB4_GENERAL_CI = a.s_area_id
                            LEFT JOIN wh_query.channel_tag AS tagname_ch ON tagname.tag_id = tagname_ch.id
                            GROUP BY tagname_ch.name;
                        """ % (db_name, start_time, end_time, s_id)
            temp_query = pd.read_sql(sql_script, conn)  # 執行上方 SQL 指令並存起來
            temp_query['月份'] = dt.strftime(dt.strptime(start_time, '%Y-%m-%d'), '%Y/%m')  # 加入月份欄位
            df = pd.concat([df, temp_query], ignore_index=True)  # 將這次的結果與先前的結果合併起來
            print(f'資料庫 {db_name} 的 {s_id} 網站在 {year_month} 的資料已運算完成。')  # 顯示目前執行進度
            start_time = end_time  # 將開始時間取代為下一個月
        else:
            break
    df.columns = ['標籤名稱', '聲量數', '月份']
    return df

### 路徑準備、迴圈撰寫

In [ ]:
demo_df.to_excel(f'C:/Users/mingyanlin/OneDrive - 意藍資訊股份有限公司/文件/01. Task/{db_name} 標籤統計202201.xlsx')

In [ ]:
dbname = "wh_fb	wh_fb_ex	wh_fb_ex_02	wh_fb_ex_03	wh_fb_ex_04	wh_fb_ex_05	wh_fb_ex_06	wh_fb_group	wh_fb_private_group	wh_fb_kol	wh_instagram	wh_backpackers	wh_bbs_01	wh_bbs_02	wh_forum_01	wh_forum_02	wh_forum_other	wh_blog_01	wh_comment_01	wh_comment_app	wh_comment_podcast	wh_plurk	wh_tiktok	wh_twitter	wh_youtube"
dbname = dbname.split("\t")
print(dbname)

In [ ]:
path = "C:/Users/mingyanlin/OneDrive - 意藍資訊股份有限公司/文件/01. Task/01. eLand/00. Data Related/頻道標籤統計/"

In [ ]:
for i in tqdm(dbname):
    demo_df = get_tag_data_by_db(i, '2022-02-01', '2022-03-01')
    newpath = path + str(i) + '標籤統計202202' + '.xlsx'
    demo_df.to_excel(newpath, index=False)